# DIAMOND - Read Simulation 1 Pbp

```
Lead     : ababaian / rce
Issue    : <github issue #>
start    : 2021 02 04
complete : 2021 XX XX
files    : NA
s3 files : s3://serratus-public/notebook/200204_sim/
```

## Introduction

What is the E-value of a false positive RdRP sequence given a huge input of data.


## EC2 Set-up script

- OS: `Amazon Linux 2 AMI (HVM) x86`
- ami: `ami-0be2609ba883822ec`
- instance: `c5.4xlarge`
- description: `"c5.4xlarge (- ECUs, 16 vCPUs, 3.4 GHz, -, 32 GiB memory, EBS only)"`
- storage: `100 GiB SSD (gp2)`
- encryption: `false`


In [ ]:
# date and version
date

In [ ]:
#!/bin/bash
# run_sapphire.sh R
# Set-up to run 8 cores
reads=$1
len=$2
R=$3

run_diamond () {
  N="$1"
  reads="$2"
  len="$3"
  R="$4"

  ./randseq_cpp $reads $len \
    | diamond blastx \
    -d rdrp1.dmnd \
    --mmap-target-index \
    --target-indexed \
    --masking 0 \
    --mid-sensitive -s 1 \
    -c1 -p1 -k1 -b 0.75 \
    -f 6 qseqid  qstart qend qlen qstrand \
         sseqid  sstart send slen \
         pident evalue cigar \
         qseq_translated full_qseq full_qseq_mate \
    > randpro/"$R$N".100Mreads."$len"nt.pro
}


# 96 runs are ~6 hours
declare -a I=($(seq 1 10  1000))
declare -a J=($(seq 10 10 1000))

for M in $(seq 0 100); do
  i=${I[M]}
  j=${J[M]}

  # run processes and store pids in array
  for N in $(seq $i $j); do
    run_diamond $N $reads $len $R &
    pids[${p}]=$!
  done

  # wait for all pids
  for pid in ${pids[*]}; do
    wait $pid
  done

done # finish M

                                     

In [ ]:
# INSTALL DIAMOND from Serratus
aws s3 cp s3://serratus-public/bin/diamond ./
chmod 755 diamond
sudo mv diamond /usr/bin/

# INSTALL randseq_cpp simulator
aws s3 cp s3://serratus-public/notebook/200204_sim/randseq_cpp ./
chmod 755 randseq_cpp

# DOWNLOAD rdrp1
aws s3 sync s3://serratus-public/seq/rdrp1/ ./

# DOWNLOAD Sapphire
aws s3 cp s3://serratus-public/notebook/200204_sim/run_sapphire.sh ./

echo done

In [ ]:
# Update script
aws s3 sync s3://serratus-public/notebook/210204_sim/ ./

In [ ]:
# Run Saphire on EC2 Instance
screen

mkdir -p randpro
chmod 755 randseq_cpp
chmod 755 run_sapphire.sh

./run_sapphire.sh 100000000 100 J

aws s3 sync randpro/ s3://serratus-public/notebook/210204_sim/randpro/
echo done
